# Standard errors for calibrated parameters: Example

Consider a simple model with two structural parameters $(\theta_1,\theta_2)$ and three reduced-form moments $(\mu_1,\mu_2,\mu_3)$. The theoretical mapping between parameters and moments is given by
$$\begin{pmatrix} \mu_1 \\ \mu_2 \\ \mu_3 \end{pmatrix} =  \begin{pmatrix} \theta_1 \\ \theta_1+\theta_2 \\ 2\theta_2 \end{pmatrix} = h(\theta_1,\theta_2).$$

We observe the noisy estimates $(\hat{\mu}_1,\hat{\mu}_2,\hat{\mu}_3) = (1.1,0.8,-0.1)$ of the true moments. The standard errors of the three empirical moments are $(\hat{\sigma}_1,\hat{\sigma}_2,\hat{\sigma}_3)=(0.1,0.2,0.05)$.

We will estimate the parameters $(\theta_1,\theta_2)$ by minimum distance, matching the model-implied moments $h(\theta_1,\theta_2)$ to the empirical moments:
$$\hat{\theta} = \text{argmin}_{\theta}\; (\hat{\mu}-h(\theta))'\hat{W}(\hat{\mu}-h(\theta)).$$

To compute standard errors for the estimated parameters, test hypotheses, and compute the efficient weight matrix $\hat{W}$, we use the formulas in [Cocci & Plagborg-Møller (2023)](https://arxiv.org/abs/2109.08109), which do not require knowledge of the correlation structure of the empirical moments.

## Define the model
We first import relevant packages and define the model and data.

In [ ]:
import numpy as np
from stderr_calibration import MinDist # Minimum distance routines

# Define moment function h(.)
G = np.array([[1,0],[1,1],[0,2]])
h = lambda theta: theta @ G.T

# Define empirical moments and their s.e.
mu = np.array([1.1,0.8,-0.1])
sigma = np.array([0.1,0.2,0.05])

# Define MinDist object used in later analysis
obj = MinDist(h,mu,moment_se=sigma)

(Note: In our simple example, we have a formula for the Jacobian of $h(\cdot)$ with respect to the parameters. This could be supplied to the `MinDist` call using the optional argument `moment_fct_deriv`. The default behavior is to compute Jacobians numerically.)

## Initial parameter estimates and standard errors
We first estimate the model using an *ad hoc* diagonal weight matrix $\hat{W}=\text{diag}(\hat{\sigma}_1^{-2},\hat{\sigma}_2^{-2},\hat{\sigma}_3^{-2})$. The numerical optimization for computing the estimates $(\hat{\theta}_1,\hat{\theta}_2)$ is started off at $(0,0)$.

In [ ]:
res = obj.fit(opt_init=np.zeros(2), eff=False) # eff=False: estimation based on ad hoc diagonal weight matrix
print('Parameter estimates')
print(res['estim'])
print('Standard errors')
print(res['estim_se'])
print('\n')
for i in range(2):
    print(f'Worst-case moment var-cov matrix for estimating theta_{i+1}')
    print(res['worstcase_varcov'][i])

(Note 1: In this simple linear example, there exists a closed-form formula for the minimum distance estimator. This formula can be supplied to the `fit()` function using the optional argument `estim_fct`.)

(Note 2: In some cases the minimum distance parameter estimate may have already been computed elsewhere. It can then be passed to the `fit()` function via the optional argument `param_estim`. The function will compute the corresponding standard errors without re-estimating the model.)

## Test of parameter restrictions
Let us test whether the parameters $\theta_1$ and $\theta_2$ equal zero.

In [ ]:
test_res = obj.test(res) # Tests are based on the "res" estimation results
print('\nt-statistics for testing individual parameters')
print(test_res['tstat'])
print('p-value of joint test')
print(test_res['joint_pval'])

Using a 5% significance level, we cannot reject that $\theta_2$ is zero individually based on its t-statistic. However, we can reject the joint hypothesis that both parameters equal zero.

Suppose we wanted to test the joint null hypothesis that $(\theta_1,\theta_2)=(1,0)$. To do this, we first reformulate it as the hypothesis that the transformed vector $r(\theta_1,\theta_2)=(\theta_1-1,\theta_2)$ has all elements equal to zero. We can then test the hypothesis as follows.

In [ ]:
r = lambda theta: theta-np.array([1,0]) # Restriction function
res_restr = obj.fit(transf=r, opt_init=res['estim'], eff=False) # Estimate the transformation r(theta)
test_res2 = obj.test(res_restr) # Test using the restriction function
print('\np-value of joint test')
print(test_res2['joint_pval'])

## Over-identification test
Since we have more moments (3) than parameters (2), we can test the over-identifying restriction. One common way of doing this in applied work is to estimate the model using only two of the moments and then checking whether the third, non-targeted moment at the estimated parameters is approximately consistent with the data.

In [ ]:
weight_mat = np.diag(np.array([1/sigma[0]**2, 1/sigma[1]**2, 0])) # Weight matrix that puts no weight on third moment
res_justid = obj.fit(opt_init=np.zeros(2), eff=False, weight_mat=weight_mat)
print('Just-identified parameter estimates')
print(res_justid['estim'])
print('Model-implied moments')
print(res_justid['moment_fit'])
print('\n')

res_overid = obj.overid(res_justid) # Over-identification test based on just-identified estimates
print('\nError in matching non-targeted moment')
print(res_overid['moment_error'][2]) # The non-targeted moment is the third one
print('Standard error')
print(res_overid['moment_error_se'][2])
print('t-statistic')
print(res_overid['tstat'][2])

Since the t-statistic is below 1.96, we can't reject the validity of the model at the 5% level.

## Efficient estimation
The above estimation results relied on an *ad hoc* diagonal weight matrix. We can compute the weight matrix that minimizes the worst-case standard errors, and then report the corresponding estimates and standard errors.

In [ ]:
res_eff = obj.fit(opt_init=np.zeros(2), eff=True) # Note: Efficient estimation (eff=True) is the default
print('Efficient parameter estimates')
print(res_eff['estim'])
print('Efficient standard errors')
print(res_eff['estim_se'])
print('\n')

for i in range(2):
    print(f'Efficient moment loadings for estimating theta_{i+1}')
    print(res_eff['moment_loadings'][:,i])

We see that $\theta_1$ is estimated off the 1st moment only, while $\theta_2$ is estimated off the 3rd moment only (up to small numerical error).

(Note: The efficient estimates are not based on a single choice of weight matrix, since the efficient weight matrix depends on the specific parameter of interest. In the background, the analysis is actually run separately for each parameter. For this reason, it is not advised to use the `test()` or `overid()` commands with efficient estimation results. These commands are better used with estimation results that correspond to a single choice of weight matrix.)

## Inference about transformed parameters
Suppose we want a confidence interval for the transformed parameter $\theta_1^2+\theta_2$. In a more realistic setting, this parameter might be some model-implied counterfactual of interest. We can do inference on transformed parameters using the `transf` argument to the `fit` function, as already used above.

In [ ]:
res_transf = obj.fit(transf=lambda theta: theta[0]**2+theta[1], opt_init=np.zeros(2)) # Efficient estimation (the default)
print('Estimated transformation')
print(res_transf['estim'])
print('Standard errors')
print(res_transf['estim_se'])

(Note: If the gradient of the transformed parameter is available, we can supply it to the `fit()` function using the optional `transf_deriv` argument.)

## More information about the variance-covariance matrix
Suppose we happen to also know that the first two empirical moments $\hat{\mu}_1$ and $\hat{\mu}_2$ are (asymptotically) independent. We can use this information to sharpen our inference about the parameters. First we define the known and unknown parts of the var-cov matrix of the empirical moments.

In [ ]:
V = np.array([[sigma[0]**2,0,np.nan],[0,sigma[1]**2,np.nan],[np.nan,np.nan,sigma[2]**2]]) # NaN values are unknown
print('Var-cov matrix of moments')
print(V)

Then we define a `MinDist` object using this var-cov matrix and apply the estimation/testing routines.

In [ ]:
obj_moreinfo = MinDist(h,mu,moment_varcov=V)
res_moreinfo = obj_moreinfo.fit(opt_init=np.zeros(2), eff=False)
print('Initial estimates')
print(res_moreinfo['estim'])
print('Standard errors')
print(res_moreinfo['estim_se'])

res_eff_moreinfo = obj_moreinfo.fit(opt_init=np.zeros(2), eff=True)
print('\nEfficient estimates')
print(res_eff_moreinfo['estim'])
print('Standard errors')
print(res_eff_moreinfo['estim_se'])

## Full-information analysis
Suppose finally that we know the entire var-cov matrix of the empirical moments. For example:

In [ ]:
V_fullinfo = sigma.reshape(-1,1) * np.array([[1,0,0.5],[0,1,-0.7],[0.5,-0.7,1]]) * sigma
print('Var-cov matrix of moments')
print(V_fullinfo)

In this full-information setting, the econometric analysis is standard ([Newey & McFadden, 1994](https://doi.org/10.1016/S1573-4412%2805%2980005-4)). The estimation and testing routines work as before.

In [ ]:
obj_fullinfo = MinDist(h,mu,moment_varcov=V_fullinfo)
res_fullinfo = obj_fullinfo.fit(opt_init=np.zeros(2), weight_mat=np.diag(sigma**(-2)), eff=False) # Diagonal weight matrix
print('Initial estimates')
print(res_fullinfo['estim'])
print('Standard errors')
print(res_fullinfo['estim_se'])

res_eff_fullinfo = obj_fullinfo.fit(opt_init=np.zeros(2), eff=True) # Efficient weight matrix
print('\nEfficient estimates')
print(res_eff_fullinfo['estim'])
print('Standard errors')
print(res_eff_fullinfo['estim_se'])

test_res_fullinfo = obj_fullinfo.test(res_eff_fullinfo)
print('\np-value for joint test that both parameters are zero')
print(test_res_fullinfo['joint_pval'])

overid_res_fullinfo = obj_fullinfo.overid(res_eff_fullinfo)
print('p-value for over-identification test')
print(overid_res_fullinfo['joint_pval'])
